##### ANN Project

* Take Churn modelling dataset
* Basic FE
  * Convert categorical to numerical
  * Standardisation
* Train
  * Apply Dropouts (disable some hidden nodes to prevent overfitting)
* Keras/Tensoflow
* Convert the model into a binary file (pickle or h5)
* Streamlit webapp 
  * Models
  * Deploy on cloud

Tensorflow new has keras in it


In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [ ]:
data = pd.read_csv('datasets/Churn_Modelling.csv')
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)
leg = LabelEncoder()
data['Gender'] = leg.fit_transform(data['Gender'])
ohe = OneHotEncoder()
gd = ohe.fit_transform(data[['Geography']])
gd.toarray()
gdf = pd.DataFrame(gd.toarray(),columns=ohe.get_feature_names_out(['Geography']))
data = pd.concat([data, gdf], axis=1)
data.drop(['Geography'], axis=1, inplace=True)
data.head()

In [ ]:
with open('datasets/label_encoded_gender.pkl', 'wb') as fp:
    pickle.dump(leg, fp)
with open('datasets/ohe_encoded_geo.pkl', 'wb') as fp:
    pickle.dump(ohe, fp)

In [ ]:
X = data.drop(['Exited'], axis=1)
y = data['Exited']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.head())
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_test

In [ ]:
with open('datasets/scaler.pkl', 'wb') as fp:
    pickle.dump(scaler, fp)

#### ANN Implementation

1. Sequential network
2. Dense parameter
3. Activation function (sigmoid, tanH, Relu, Leaky Relu)
4. Optimizer (Back Propagation)
5. Loss Function
6. Metrics

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [ ]:
ann = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
ann.summary()

In [ ]:
#Compile Model
ann.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=['accuracy'])

In [ ]:
#Logs for analysis
log_dir='logs/fit'
tf_cb = TensorBoard(log_dir=log_dir)


In [ ]:
# Setup early stopping
es_cb = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
## Train the model
history = ann.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[tf_cb, es_cb])

In [ ]:
ann.save('datasets/ann_churn.h5')

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=logs/fit